## data transform

In [15]:
from data_transformer import DataTransformer
import pandas as pd 
import numpy as np
import tensorflow as tf

2023-05-08 14:52:42.497125: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [6]:
rawdata = pd.read_csv('adult.csv')
cat_cols = [ 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
log_cols = []
mixed_cols= {'capital-loss':[0.0],'capital-gain':[0.0]}
num_cols = ['age', 'fnlwgt','educational-num','hours-per-week']
target_col = 'income'

In [7]:
transformer = DataTransformer(rawdata, cat_cols, num_cols,mixed_cols, log_cols, target_col)

In [8]:
#preprocess data
df = transformer.transformData()

In [9]:
transformer.fit(df)

In [10]:
transformed_data = transformer.transform(df)

In [11]:
output_info = transformer.output_info_list

In [12]:
from data_sampler import DataSampler
sampler = DataSampler(transformed_data, output_info)

## build generator

In [13]:
# obtaining the desired height/width for generating square images from the generator network that can be converted back to tabular domain 
data_dim = transformer.output_dimensions		
sides = [4, 8, 16, 24, 32]
col_size_g = data_dim
for i in sides:
    if i * i >= col_size_g:
        gside = i
        break

num_channels = 64
#num_channels=
# computing the dimensionality of hidden layers
layer_dims = [(1, gside), (num_channels, gside // 2)]

while layer_dims[-1][1] > 3 and len(layer_dims) < 4:
    layer_dims.append((layer_dims[-1][0] * 2, layer_dims[-1][1] // 2))

In [19]:
#create a Batch Normalization layer with custom weight and bias initializers
def custom_init_weights(shape, dtype=None):
    return tf.random.normal(shape, mean=1.0, stddev=0.02, dtype=dtype)

def custom_init_bias(shape, dtype=None):
    return tf.zeros(shape, dtype=dtype)

In [20]:
random_dim = 100
inp = tf.keras.layers.Input(shape=(1,1, random_dim + sampler.n_opt))
x = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=2, strides=1, padding='valid', use_bias=False)(inp)
x = tf.keras.layers.BatchNormalization(beta_initializer=custom_init_bias,gamma_initializer=custom_init_weights)(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=4, strides=2, padding='same', use_bias=True)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=4, strides=2, padding='same', use_bias=True)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
out = tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=4, strides=2, padding='same', use_bias=True)(x)
generator = tf.keras.models.Model(inp, out)

In [21]:
generator.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1, 1, 258)]       0         
_________________________________________________________________
conv2d_transpose_8 (Conv2DTr (None, 2, 2, 256)         264192    
_________________________________________________________________
batch_normalization_6 (Batch (None, 2, 2, 256)         1024      
_________________________________________________________________
re_lu_6 (ReLU)               (None, 2, 2, 256)         0         
_________________________________________________________________
conv2d_transpose_9 (Conv2DTr (None, 4, 4, 128)         524416    
_________________________________________________________________
batch_normalization_7 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
re_lu_7 (ReLU)               (None, 4, 4, 128)         0   

: 